In [193]:
import pandas as pd
import numpy as np
from qiime2 import Artifact, Visualization

In [1]:
cd /Users/johnsterrett/Research-Projects/Boulder/Chemobrain/200608_Elsayed_Lowry-1101

/Users/johnsterrett/Research-Projects/Boulder/Chemobrain/200608_Elsayed_Lowry-1101


In [2]:
ls

ANCOM-L2-chemo.qzv
ANCOM-L2-hcvsbc.qzv
ANCOM-L3-chemo.qzv
ANCOM-L3-hcvsbc.qzv
ANCOM-L6-hcvsbc.qzv
Akkermansia-boxplots.pdf
CESD-LEfSe.pdf
Cat-meta-no-datetime.txt
Categorical-meta-noNA.tsv
Categorical-meta-v3.tsv
Chemo-LEfSe.pdf
CompletedJobInfo.xml*
FACT-LEfSe.pdf
GenerateFASTQRunStatistics.xml*
HLR-abundances-nona.csv
HLR-abundances.csv
InterOp/
L2p/
L2p-filtered.qza
L3p/
L3p-filtered.qza
L6p/
L6p-filtered.qza
RunInfo.xml*
age-cesd-fact-boxplots.pdf
akkermansia-time-all.pdf
akkermansia-time.pdf
alpha-rarefaction.qzv
alpha-sigcorr-qiime.py
alpha-sigcorr-qiime.py.1
alphadiv-bc-cesd-trunc.png
alphadiv-bc-cesd-trunc.svg
alphadiv-bc-cesd.png
alphadiv-bc-cesd.svg
beta-rarefaction.qzv
beta-sig-corr-qiime.py
beta_corr_results_pearson/
biplot-L2.qza
biplot-L2.qzv
biplot-bray-L2.qza
biplot-bray-L2.qzv
bray_curtis_corr_results.csv
bray_curtis_sig_results.csv
by_phylum.csv
clr_transformed_sharedtaxa.csv
clr_transformed_sharedtaxa_2.csv
core-diversity-results/
core-diversity-results-noNA/
dada2-r

In [3]:
! wget https://raw.githubusercontent.com/knightlab-analyses/bloom-analyses/master/data/bloom_taxa.txt

--2021-07-15 10:57:35--  https://raw.githubusercontent.com/knightlab-analyses/bloom-analyses/master/data/bloom_taxa.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3055 (3.0K) [text/plain]
Saving to: ‘bloom_taxa.txt’

bloom_taxa.txt      100%[===================>]   2.98K  --.-KB/s    in 0s      

2021-07-15 10:57:35 (10.4 MB/s) - ‘bloom_taxa.txt’ saved [3055/3055]



In [105]:
bloomdf = pd.read_csv("bloom_taxa.txt", sep="\t")

bloomdf.columns = ['featureid', 'Storage Change', 'Freeze-Thaw change', 'Taxonomy']

#format taxonomy
bloomdf["Taxonomy"] = bloomdf["Taxonomy"].apply(lambda x: "d" + x[1:])
bloomdf["Taxonomy"] = bloomdf["Taxonomy"].apply(lambda x: "; ".join(x.split(";")))

In [126]:
bloomdf.head()

,featureid,Storage Change,Freeze-Thaw change,Taxonomy,Taxonomy (genus)
0,691656f33e727f8857f5c9ee40ed05b1,280.77,56.41,d__Bacteria; p__Proteobacteria; c__Gammaproteo...,d__Bacteria; p__Proteobacteria; c__Gammaproteo...
1,41f1c6e4d1e4cc653df22555bfab5796,160.78,17.81,d__Bacteria; p__Proteobacteria; c__Gammaproteo...,d__Bacteria; p__Proteobacteria; c__Gammaproteo...
2,ece1af985b63ebccd2833e9b5f0432e3,138.78,0.85,d__Bacteria; p__Proteobacteria; c__Gammaproteo...,d__Bacteria; p__Proteobacteria; c__Gammaproteo...
3,04195686f2b70585790ec75320de0d6f,97.38,7.03,d__Bacteria; p__Proteobacteria; c__Gammaproteo...,d__Bacteria; p__Proteobacteria; c__Gammaproteo...
4,df3f8cdac76c62baa98469c33b82d8df,78.79,2.66,d__Bacteria; p__Proteobacteria; c__Gammaproteo...,d__Bacteria; p__Proteobacteria; c__Gammaproteo...


In [106]:
taxonomy = Artifact.load("taxonomy-silva.qza").view(pd.DataFrame)
taxonomy = taxonomy.reset_index().set_index("Taxon")

In [107]:
taxonomy.index[np.where([x in bloomdf["Taxonomy"].values for x in taxonomy.index])]

Index([], dtype='object', name='Taxon')

**Can see above that it isn't picking up these taxa when classified down to the species level**

In [111]:
bloomdf["Taxonomy (genus)"] = bloomdf["Taxonomy"].apply(lambda x: x.split("; s__")[0])

In [124]:
tofilt = taxonomy.index[np.where([x in bloomdf["Taxonomy (genus)"].values for x in taxonomy.index])]

In [125]:
set(tofilt)

{'d__Bacteria; p__Firmicutes; c__Bacilli; o__Bacillales; f__Bacillaceae; g__Bacillus',
 'd__Bacteria; p__Firmicutes; c__Bacilli; o__Lactobacillales; f__Lactobacillaceae; g__Lactobacillus',
 'd__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Pseudomonadales; f__Pseudomonadaceae; g__Pseudomonas'}

In [116]:
set(bloomdf["Taxonomy (genus)"])-set(tofilt)

{'d__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides',
 'd__Bacteria; p__Firmicutes; c__Bacilli; o__Bacillales; f__Planococcaceae; g__Sporosarcina',
 'd__Bacteria; p__Firmicutes; c__Bacilli; o__Bacillales; f__Staphylococcaceae; g__Staphylococcus',
 'd__Bacteria; p__Firmicutes; c__Bacilli; o__Lactobacillales; f__Lactobacillaceae; g__',
 'd__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae',
 'd__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__',
 'd__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Citrobacter',
 'd__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Escherichia',
 'd__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Klebsiella',
 'd__Bacteria; p__Proteobacteria; c__Gammap

## SILVA assigned taxonomy isn't showing up, so I'm going to reclassify based on Greengenes

In [129]:
mkdir reanalysis

In [132]:
!qiime feature-classifier classify-sklearn \
--i-reads dada2-results/representative_sequences.qza \
--i-classifier gg-13-8-99-515-806-nb-classifier.qza \
--o-classification reanalysis/taxonomy-gg.qza

Saved FeatureData[Taxonomy] to: reanalysis/taxonomy-gg.qza


In [141]:
taxonomy = Artifact.load("reanalysis/taxonomy-gg.qza").view(pd.DataFrame)
taxonomy = taxonomy.reset_index().set_index("Taxon")

In [145]:
bloomdf = pd.read_csv("bloom_taxa.txt", sep="\t")
bloomdf.columns = ['featureid', 'Storage Change', 'Freeze-Thaw change', 'Taxonomy']
bloomdf["Taxonomy"] = bloomdf["Taxonomy"].apply(lambda x: "; ".join(x.split(";")))

### We're able to pick out these 7 that are in the AGP bloomed df

In [272]:
tofilt = taxonomy.index[np.where( [(x in bloomdf["Taxonomy"].values) for x in taxonomy.index] )]
set(tofilt)

{'k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__uniformis',
 'k__Bacteria; p__Firmicutes; c__Bacilli; o__Bacillales; f__Bacillaceae; g__Bacillus; s__clausii',
 'k__Bacteria; p__Firmicutes; c__Bacilli; o__Lactobacillales; f__Lactobacillaceae; g__Lactobacillus; s__zeae',
 'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae',
 'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Providencia; s__',
 'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Pseudomonadales; f__Pseudomonadaceae; g__Pseudomonas; s__',
 'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Pseudomonadales; f__Pseudomonadaceae; g__Pseudomonas; s__fragi'}

In [273]:
notfound = list(set(bloomdf["Taxonomy"])-set(tofilt))
notfound

['k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__; s__',
 'k__Bacteria; p__Firmicutes; c__Bacilli; o__Bacillales; f__Planococcaceae; g__Sporosarcina; s__ginsengi',
 'k__Bacteria; p__Firmicutes; c__Bacilli; o__Lactobacillales; f__Lactobacillaceae; g__; s__',
 'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Klebsiella; s__',
 'k__Bacteria; p__Firmicutes; c__Bacilli; o__Bacillales; f__Staphylococcaceae; g__Staphylococcus; s__',
 'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Escherichia; s__coli',
 'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Trabulsiella; s__farmeri',
 'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Citrobacter; s__',
 'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobac

In [274]:
entero = "k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae"
taxonomy.index[np.where([x.startswith(entero) for x in taxonomy.index])]

Index(['k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Shigella',
       'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Shigella',
       'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Shigella',
       'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Shigella',
       'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae',
       'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Shigella',
       'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Shigella',
       'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae',
       'k__Bacteria; p__Proteobacteria; c_

In [275]:
notfound_genus = [x.split("; s__")[0] for x in notfound]
for tax in notfound_genus:
    print(taxonomy.index[np.where( [(x.startswith(tax)) for x in taxonomy.index] )])

Index(['k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Shigella',
       'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Shigella',
       'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Shigella',
       'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Shigella',
       'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Shigella',
       'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Shigella',
       'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Shigella',
       'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Proteus; s__vulgaris',
    

In [277]:
tofilt = list(tofilt)
tofilt += ['k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Morganella',
           'k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacteriales; f__Enterobacteriaceae; g__Proteus; s__vulgaris']

In [279]:
tofilt = pd.Index(tofilt)

## Actually filtering

In [280]:
tofilt_df = taxonomy.loc[tofilt,"Feature ID"]
#put columns in order
tofilt_df = tofilt_df.reset_index().reindex(["Feature ID", "Taxon"], axis=1)

In [281]:
print(len(tofilt_df["Feature ID"]))
print(len(tofilt_df["Feature ID"].unique()))

905
53


In [282]:
bloomed_sequence_ids = pd.Series(tofilt_df["Feature ID"].unique(),name="Feature ID")

In [283]:
bloomed_sequence_ids.to_csv("reanalysis/bloomed_sequence_ids.tsv", sep="\t", index=False)

In [284]:
!qiime feature-table filter-features \
--i-table noMito_noChloro-filtered-table.qza \
--m-metadata-file reanalysis/bloomed_sequence_ids.tsv \
--p-exclude-ids \
--o-filtered-table reanalysis/nobloom-table.qza

Saved FeatureTable[Frequency] to: reanalysis/nobloom-table.qza


In [285]:
viz = Visualization.load("noMito_noChloro-filtered-table.qzv")
viz

<visualization: Visualization uuid: e19c142b-48a2-4b33-9926-d5a60bb10022>

In [286]:
! qiime feature-table summarize \
--i-table reanalysis/nobloom-table.qza \
--m-sample-metadata-file Categorical-meta-v3.tsv \
--o-visualization reanalysis/nobloom-table.qzv

Saved Visualization to: reanalysis/nobloom-table.qzv


In [287]:
viz = Visualization.load("reanalysis/nobloom-table.qzv")
viz

<visualization: Visualization uuid: bbf5a825-dc2e-41e8-afcd-aaa503a9a111>

In [288]:
! qiime feature-table filter-samples \
--i-table reanalysis/nobloom-table.qza \
--m-metadata-file expr-group-metadata.tsv \
--o-filtered-table reanalysis/nobloom-filt-table.qza

Saved FeatureTable[Frequency] to: reanalysis/nobloom-filt-table.qza


In [289]:
! qiime taxa barplot \
--i-table reanalysis/nobloom-filt-table.qza \
--i-taxonomy reanalysis/taxonomy-gg.qza \
--m-metadata-file expr-group-metadata.tsv \
--o-visualization reanalysis/taxa-barplot-gg.qzv

Saved Visualization to: reanalysis/taxa-barplot-gg.qzv


In [290]:
viz = Visualization.load("reanalysis/taxa-barplot-gg.qzv")
viz

<visualization: Visualization uuid: ede6bcbc-5307-4fb6-82f3-d30fb0d68beb>

In [204]:
ls rea

ANCOM-L2-chemo.qzv
ANCOM-L2-hcvsbc.qzv
ANCOM-L3-chemo.qzv
ANCOM-L3-hcvsbc.qzv
ANCOM-L6-hcvsbc.qzv
Akkermansia-boxplots.pdf
CESD-LEfSe.pdf
Cat-meta-no-datetime.txt
Categorical-meta-noNA.tsv
Categorical-meta-v3.tsv
Chemo-LEfSe.pdf
CompletedJobInfo.xml*
FACT-LEfSe.pdf
GenerateFASTQRunStatistics.xml*
HLR-abundances-nona.csv
HLR-abundances.csv
InterOp/
L2p/
L2p-filtered.qza
L3p/
L3p-filtered.qza
L6p/
L6p-filtered.qza
RunInfo.xml*
age-cesd-fact-boxplots.pdf
akkermansia-time-all.pdf
akkermansia-time.pdf
alpha-rarefaction.qzv
alpha-sigcorr-qiime.py
alpha-sigcorr-qiime.py.1
alphadiv-bc-cesd-trunc.png
alphadiv-bc-cesd-trunc.svg
alphadiv-bc-cesd.png
alphadiv-bc-cesd.svg
beta-rarefaction.qzv
beta-sig-corr-qiime.py
beta_corr_results_pearson/
biplot-L2.qza
biplot-L2.qzv
biplot-bray-L2.qza
biplot-bray-L2.qzv
bloom_taxa.txt
bloomed_sequence_ids.tsv
bray_curtis_corr_results.csv
bray_curtis_sig_results.csv
by_phylum.csv
clr_transformed_sharedtaxa.csv
clr_transformed_sharedtaxa_2.csv
core-diversity-resul